In [1]:
from nltk import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer as wnl
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from afinn import Afinn
import nltk, string, glob
import gensim
import itertools
import re
import csv
import scipy
import warnings
import numpy as np
import pandas as pd
warnings.simplefilter(action='ignore', category=FutureWarning)

[nltk_data] Downloading package stopwords to /Users/stiso/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/stiso/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/stiso/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
exts = ['bio', 'wiki']

In [10]:
afn = Afinn()
afn.score('bit')

0.0

In [11]:
bio = gensim.models.KeyedVectors.load_word2vec_format(
    "../data/bioASQmodel.txt", 
    binary=False, 
    unicode_errors='ignore'
)
wiki = gensim.models.KeyedVectors.load_word2vec_format(
    "../data/enwiki_20180420_300d.txt", 
    binary=False, 
    unicode_errors='ignore'
)

models = [bio, wiki]

In [ ]:
m.most_similar(positive='woman', topn=5)

In [25]:
# most similar words to 'woman' and 'man'
word = 'more'
print(models[1].similarity(word, 'man'))
print(models[1].similarity(word, 'woman'))

0.25159326
0.23208015


In [5]:
#################################################
# Initialize, config & define helpful functions #
#################################################

translator = str.maketrans('', '', string.punctuation.replace('-', '')) #filters punctuation except dash
lemmatizeCondition = 1
lemmatizer = wnl()
stop_words = nltk.corpus.stopwords.words('english')

# Function for finding index of words of interest, like 'references'

def find(target):
    for i, word in enumerate(sents):
        try:
            j = word.index(target)
        except ValueError:
            continue
        yield i

# Function for handling the input for gensim word2vec

class FileToSent(object):
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, 'r'):
            ll = line.strip().split(",")
            ll = [''.join(c for c in s if c not in string.punctuation) for s in ll]
            ll = [num.strip() for num in ll]
            yield ll

In [6]:
###################################################
# Read in .txt file(s) from a specified directory #
###################################################

evals = pd.read_csv('../data/psom_evals.csv')
evals.head()

/Users/stiso/anaconda2/envs/psom/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (11,12,13,69,70) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Evaluatee,StudentName,FormID,Summary,Question,Cid,S_URM,S_SRS.Calc.Race.Desc,Schedule.Start.Date,Answer.Text_Detail,...,F_nh_black_std,F_nh_black_lb,F_nh_black_ub,F_nh_white_mean,F_nh_white_std,F_nh_white_lb,F_nh_white_ub,F_race,F_gender50,F_gender70
0,"AAKHUS, ERIN","HERNANDO, MARY",e2d1755825d32c6be44b8bcf098d20f935a0a665c99a23...,,Areas for improvement/Additional comments,MED,Y,Multiple ethnicities selected,9/11/2017,No concerns for her level of training. I only ...,...,0.044043,0.002035,0.002561,0.931659,0.046282,0.733805,0.747535,nh_white,M,M
1,"AAKHUS, ERIN","HERNANDO, MARY",e2d1755825d32c6be44b8bcf098d20f935a0a665c99a23...,,Strengths/Summary,MED,Y,Multiple ethnicities selected,9/11/2017,Mary Ann impressed me with her thorough histor...,...,0.044043,0.002035,0.002561,0.931659,0.046282,0.733805,0.747535,nh_white,M,M
2,"AAMODT, WHITLEY","MEZOCHOW, GABRIELLE",bf3434488ad0a3be485d804cdba2d6aeee56c226519712...,,Strengths/Summary,NEU,N,"White, Non-Hispanic",3/26/2018,Gabby did an excellent job on the neurology se...,...,0.164404,0.161161,0.179011,0.160745,0.111224,0.020521,0.021742,nh_black,W,W
3,"AAMODT, WHITLEY","RECH, ANDREW",43cf5ed1d5f3a7ead23e25da8a7d5b0bee2540c12cdf3b...,,Strengths/Summary,NEU,N,"White, Non-Hispanic",2/26/2018,Andrew did an outstanding job on the PPMC neur...,...,0.164404,0.161161,0.179011,0.160745,0.111224,0.020521,0.021742,nh_black,W,W
4,"AAMODT, WHITLEY","O''KEEFE, RYAN",1f637c27634cc0b947be3dfcf9998f50fb47d0185e71a4...,,Strengths/Summary,NEU,N,"White, Non-Hispanic",9/24/2018,Ryan did an excellent job on the HUP neurology...,...,0.164404,0.161161,0.179011,0.160745,0.111224,0.020521,0.021742,nh_black,W,W


In [7]:
####################
# Clean, lemmatize #
####################
w_words = ['she', 'daughter', 'hers', 'her', 'mother', 'woman', 'girl', 'herself', 
           'female', 'sister', 'daughters', 'mothers', 'women', 'girls', 'females',
          'sisters', 'aunt', 'aunts', 'niece', 'nieces']
m_words = ['he', 'son', 'his', 'him', 'father', 'man', 'boy', 'himself', 
           'male', 'brother', 'son', 'father', 'men', 'boys', 'males',
          'brothers', 'uncles', 'uncle', 'nephew', 'nephews']

afn = Afinn()

for ID,row in evals.iterrows(): # loop through papers
    totalWords = []

    text = row['Answer.Text_Detail']
    text = re.sub("\u2013|\u2014", "-", str(text))  # Replace em-dashes
    sents = sent_tokenize(text)  # Split into sentences
    sents = [word_tokenize(s) for s in sents]
    sents = [[w.translate(translator) for w in s] for s in sents]  # filter punctuation
    sents = [[re.sub(r'^[-+]?[0-9]*[\.\-]?[0-9]+$', 'numeric', w) for w in s] for s in sents]  # replace all numerals with the holder "number"
    sents = [[w for w in s if re.search('[^a-zA-Z-0-9-]+', w) is None] for s in sents]  # trips everything but alphanumeric
    sents = [[w.lower() for w in s] for s in sents]  # make lower case
    sents = [s for s in sents if len(s) > 0]  # remove empty lines
    sents = [[w for w in s if not w in stop_words] for s in sents]  # filter stop words
    sents = [[w for w in s if len(w) > 1] for s in sents]  # filters out variables, etc
    sents = [[w for w in s if len(w) > 2] for s in sents]  # filters out variables, etc
    sents = [[w for w in s if len(w) > 3] for s in sents]  # filters out variables and abbreviations
    sents = [s for s in sents if len(s) > 0]  # remove empty lines
    words = [[lemmatizer.lemmatize(w) for w in s if lemmatizeCondition == 1] for s in sents]  # lemmatize
    words = list(itertools.chain.from_iterable(words))  # join list of lists
    totalWords.append(words)

    # append unique words in the whole corpus
    my_words = list(set(list(itertools.chain.from_iterable(totalWords))))  
    
    # remove names
    name = row.StudentName.lower().split(' ')
    my_words = [w for w in my_words if w.lower() not in name]
    words = [w for w in words if w.lower() not in name]
    evals.loc[ID,'text_len'] = len(words)
    
    # sentiment analysis
    scores = [afn.score(w) for w in words]

    # loop through models
    for i,m in enumerate(models):
        # filter out words not in model
        my_words = [word for word in my_words if word in m]
        
        # filter out words of interest
        my_words = [word for word in my_words if word not in w_words]
        my_words = [word for word in my_words if word not in m_words]
        
        # Make a list of all word-to-word distances [each as a tuple of (word1,word2,dist)]
        sims_w = []
        sims_m = []
        
        # Find similarity distances between each word pair for current year
        for word1 in my_words:
            sim = []
            for word2 in w_words:
                cosine_similarity = m.similarity(word1, word2)
                sim.append(cosine_similarity)
            sims_w.append(np.mean(sim))
        for word1 in my_words:
            sim = []
            for word2 in m_words:
                cosine_similarity = m.similarity(word1, word2)
                sim.append(cosine_similarity)
            sims_m.append(np.mean(sim))

        if len(sims_w) > 0:
            # gender coded
            evals.loc[ID,f'mean_wom_{exts[i]}'] = np.mean(sims_w)
            evals.loc[ID,f'med_wom_{exts[i]}'] = np.median(sims_w)
            evals.loc[ID,f'max_wom_{exts[i]}'] = np.max(sims_w)
            evals.loc[ID,f'mean_man_{exts[i]}'] = np.mean(sims_m)
            evals.loc[ID,f'med_man_{exts[i]}'] = np.median(sims_m)
            evals.loc[ID,f'max_man_{exts[i]}'] = np.max(sims_m)
    # sentiment
    if len(scores) > 1:
        evals.loc[ID,'sum_sent'] = sum(scores)
        evals.loc[ID,'mean_sent'] = np.mean(scores)
        if sum(scores) > 0:
            evals.loc[ID,'sent'] = 'positive'
        else:
            evals.loc[ID,'sent'] = 'negative'
    else:
        evals.loc[ID,'sum_sent'] = np.nan
        evals.loc[ID,'mean_sent'] = np.nan
        

In [8]:
evals.to_csv("../data/psom_evals_nlp.csv")
evals.head()

,Evaluatee,StudentName,FormID,Summary,Question,Cid,S_URM,S_SRS.Calc.Race.Desc,Schedule.Start.Date,Answer.Text_Detail,...,max_man_bio,mean_wom_wiki,med_wom_wiki,max_wom_wiki,mean_man_wiki,med_man_wiki,max_man_wiki,sum_sent,mean_sent,sent
0,"AAKHUS, ERIN","HERNANDO, MARY",e2d1755825d32c6be44b8bcf098d20f935a0a665c99a23...,,Areas for improvement/Additional comments,MED,Y,Multiple ethnicities selected,9/11/2017,No concerns for her level of training. I only ...,...,0.277099,0.186815,0.180086,0.270845,0.175862,0.182334,0.231065,6.0,0.545455,positive
1,"AAKHUS, ERIN","HERNANDO, MARY",e2d1755825d32c6be44b8bcf098d20f935a0a665c99a23...,,Strengths/Summary,MED,Y,Multiple ethnicities selected,9/11/2017,Mary Ann impressed me with her thorough histor...,...,0.281561,0.189576,0.184051,0.295312,0.185006,0.181844,0.297524,8.0,0.242424,positive
2,"AAMODT, WHITLEY","MEZOCHOW, GABRIELLE",bf3434488ad0a3be485d804cdba2d6aeee56c226519712...,,Strengths/Summary,NEU,N,"White, Non-Hispanic",3/26/2018,Gabby did an excellent job on the neurology se...,...,0.315162,0.196609,0.197174,0.290263,0.199983,0.209590,0.281945,17.0,0.320755,positive
3,"AAMODT, WHITLEY","RECH, ANDREW",43cf5ed1d5f3a7ead23e25da8a7d5b0bee2540c12cdf3b...,,Strengths/Summary,NEU,N,"White, Non-Hispanic",2/26/2018,Andrew did an outstanding job on the PPMC neur...,...,0.303533,0.192020,0.190689,0.284315,0.191886,0.199738,0.311979,11.0,0.224490,positive
4,"AAMODT, WHITLEY","O''KEEFE, RYAN",1f637c27634cc0b947be3dfcf9998f50fb47d0185e71a4...,,Strengths/Summary,NEU,N,"White, Non-Hispanic",9/24/2018,Ryan did an excellent job on the HUP neurology...,...,0.303533,0.181080,0.183204,0.280025,0.186723,0.189369,0.266286,9.0,0.250000,positive


In [9]:
evals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57734 entries, 0 to 57733
Data columns (total 87 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Evaluatee             57734 non-null  object 
 1   StudentName           57734 non-null  object 
 2   FormID                57734 non-null  object 
 3   Summary               57734 non-null  object 
 4   Question              57734 non-null  object 
 5   Cid                   57734 non-null  object 
 6   S_URM                 54870 non-null  object 
 7   S_SRS.Calc.Race.Desc  55340 non-null  object 
 8   Schedule.Start.Date   57734 non-null  object 
 9   Answer.Text_Detail    56708 non-null  object 
 10  StudentID             57734 non-null  int64  
 11  Instructor_ID         46854 non-null  object 
 12  F_Gender              24762 non-null  object 
 13  F_Race_Ethnicity      24762 non-null  object 
 14  F_Afr.Amer            24762 non-null  float64
 15  F_Asian            